In [140]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [141]:
print("Hello World")

Hello World


In [142]:
# first we gotta fetch the website
#website is: https://www.ccny.cuny.edu/registrar/fall
URL = 'https://www.ccny.cuny.edu/registrar/fall'
res = requests.get(url=URL)
print(res.headers)

{'Connection': 'keep-alive', 'Content-Length': '26697', 'Cache-Control': 'max-age=1800, public', 'Content-Encoding': 'gzip', 'Content-Language': 'en', 'Content-Type': 'text/html; charset=UTF-8', 'Etag': 'W/"1759352213"', 'Expires': 'Sun, 19 Nov 1978 05:00:00 GMT', 'Last-Modified': 'Wed, 01 Oct 2025 20:56:53 GMT', 'Server': 'nginx', 'Strict-Transport-Security': 'max-age=31622400', 'X-Content-Type-Options': 'nosniff', 'X-Drupal-Cache': 'MISS', 'X-Drupal-Dynamic-Cache': 'MISS', 'X-Frame-Options': 'SAMEORIGIN', 'X-Generator': 'Drupal 10 (https://www.drupal.org)', 'X-Pantheon-Styx-Hostname': 'styx-fe3-a-6966ffb76d-sqsg8', 'X-Styx-Req-Id': '28b6f945-9f09-11f0-be4e-4642d4bfde44', 'Date': 'Wed, 01 Oct 2025 23:28:50 GMT', 'X-Served-By': 'cache-chi-kigq8000114-CHI, cache-ewr-kewr1740045-EWR', 'X-Cache': 'MISS, HIT', 'X-Cache-Hits': '0, 0', 'X-Timer': 'S1759361330.177104,VS0,VE11', 'Vary': 'Accept-Encoding, Cookie, Cookie, Cookie', 'Age': '1705', 'Accept-Ranges': 'bytes', 'Via': '1.1 varnish, 1.1

In [143]:
#now that we fetched the site, we want to extract the html with beautifulsoup
html_data = res.content
soup = BeautifulSoup(html_data, 'html.parser')
# print(soup.prettify())

#now that we have extracted the html we find where most of the data for the list is
tr_tag = soup.find_all('tr')
#returns a list of the entire list with each index containing where <tr> upto </tr>

#we will now iterate to build up a 
#we have two ways to construct our DataFrame: List of Dicts or Dict of Lists
# list of dicts allows us to construct it in such a way that each row is created with its relecant data
"""
[
    {
        "date": "Aug, 1",
        "dow": "Mon",
        "text": "The great noticing":
    }
]

"""

'\n[\n    {\n        "date": "Aug, 1",\n        "dow": "Mon",\n        "text": "The great noticing":\n    }\n]\n\n'

In [170]:
#using the .string attribute allows us to get the string between tags

cal_yr = soup.title.string
yr = re.search(r'\d+', cal_yr)
if yr: yr = yr.group(0) #extracting the year from the title with regex


def find_dt_range_nums(date_str, strt=True) -> int: 
    if strt:
        return int(date_str[date_str.index('-') - 3:date_str.index('-') - 1]) # remember on slicing :end is not included
    return int(date_str[date_str.index('-') + 2:])


print(tr_tag[2].find_all('td')[0].find('p').string) #some ch

df_prep = []
for tr in tr_tag[1:]: 
    tr_chunk = tr.find_all('td') # chunking is better done over the td tags since each seperate label has them
    df_row = {
        "date":  tr_chunk[0].string or tr_chunk[0].find('p').string, #binary short circuit evaluation helps make this effective without crazy checks
        "dow": tr_chunk[1].string or tr_chunk[1].find('p').string,
        "text": tr_chunk[2]
    }
        
    df_prep.append(df_row)

August 18


In [173]:
#finally created the dataframe

class_sched_df = pd.DataFrame(df_prep)
class_sched_df.set_index(keys='date', inplace=True)
print(class_sched_df)
print(class_sched_df.shape)

                                    dow  \
date                                      
August 01                        Sunday   
August 18                     Wednesday   
August 24                       Tuesday   
August 25                     Wednesday   
August 25 - 31      Wednesday - Tuesday   
August 26                      Thursday   
August 28                      Saturday   
August 31                       Tuesday   
September 01                  Wednesday   
September 03 - 08    Friday - Wednesday   
September 06                     Monday   
September 09                   Thursday   
September 14                    Tuesday   
September 15                  Wednesday   
September 15 - 16  Wednesday - Thursday   
September 23                   Thursday   
September 24                     Friday   
October 01                       Friday   
October 08                       Friday   
October 11                       Monday   
November 01                      Monday   
November 02